<a href="https://colab.research.google.com/github/talibaisi/CoLab/blob/master/engine_v3_0_0_0_IoT2023_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import csv
import time
import joblib

import shap
import matplotlib.pyplot as plt
import os
import logging


#GPU
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import skew, kurtosis

import resource

from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, ClassifierMixin

import pandas as pd
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from torch.utils.data.dataset import random_split
from torch.nn import BCELoss, Sigmoid
from torch.optim import Adam

import sqlite3
import numpy as np
import logging,sys
import pandas as pd
from joblib import dump, load
import smtplib
from email.mime.text import MIMEText
from sklearn.ensemble import StackingClassifier,BaggingClassifier, GradientBoostingClassifier
#Machine Learning packages import
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import scale

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

import tensorflow as tf
from sklearn.model_selection import GridSearchCV

import datetime

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import IsolationForest
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import matthews_corrcoef
import torch.nn.functional as F
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel


#Global parameters
#==================================================
CIC_IoT_table_name = 'CIC-IoT-2023'

# Define batch size
CIC_IoT_Reading_Batch_Size = 300000
DATAFAME_TO_SQLITE_BATCH_Size = 1000
K_NO_OF_FEATURES_TO_SELECT = 20 #10  # Number of features to select
isRunningOnLinux = False
IDSLOG_FILE_MODE = 'a' # put 'w' if you want to start new fresh IDSLog.log rather than append to existing one
n_Total_read_rows_from_CIC_IOT = None # put 'None' incase to read all records from CIC IoT csv (linux run)
RESAMPLING_RATIO = 100
RESAMPLE_ALGORITHM = 'SMOTE' #'ADASYN' #USE 'NONE' in case no sampling


In [ ]:
#==================================================
def calculate_g_mean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    g_mean = np.sqrt(sensitivity * specificity)
    return g_mean

#Function to calculate and log skewness and kurtosis
def log_skewness_kurtosis(data, feature_names):
    logging.info("")
    logging.info("***********************************************************************")
    logging.info("log_skewness_kurtosis starts")
    logging.info("")

    print("")
    print("***********************************************************************")
    print("log_skewness_kurtosis starts")
    print("")


    skewness = skew(data, axis=0)
    kurt = kurtosis(data, axis=0)
    for i, feature in enumerate(feature_names):
        logging.info(f"Feature: {feature} - Skewness: {skewness[i]}, Kurtosis: {kurt[i]}")
        print(f"Feature: {feature} - Skewness: {skewness[i]}, Kurtosis: {kurt[i]}")

    logging.info("log_skewness_kurtosis ends")
    logging.info("***********************************************************************")

    print("log_skewness_kurtosis ends")
    print("***********************************************************************")

def ResampleDataFrame(df_all, classCol="Label",commentOnDataframe=""):
        # Assuming 'label' is the column indicating benign (0) or attack (1)
        # Split the data into features (X) and labels (y)
        X = df_all.drop(classCol, axis=1)
        y = df_all[classCol]

        # Resample the imbalanced dataset using SMOTE or ADASYN
        # Choose one of the following resampling techniques
        # oversampler = SMOTE(sampling_strategy=0.5)  # SMOTE
        logging.info('')
        logging.info('****************************************************************')
        logging.info('RESAMPLE ('+commentOnDataframe+') using '+RESAMPLE_ALGORITHM+' algorithm with ratio '+str(RESAMPLING_RATIO))
        logging.info('****************************************************************')
        logging.info('')

        logging.info('y.unique() = '+str(y.unique()))
        logging.info('y.value_counts() = '+str(y.value_counts()))
        logging.info('')


        print('')
        print('****************************************************************')
        print('RESAMPLE ('+commentOnDataframe+') using '+RESAMPLE_ALGORITHM+' algorithm with ratio '+str(RESAMPLING_RATIO))
        print('****************************************************************')
        print('')

        print('y.unique() = '+str(y.unique()))
        print('y.value_counts() = '+str(y.value_counts()))
        print('')

        if RESAMPLE_ALGORITHM == 'SMOTE':
            oversampler = SMOTE(sampling_strategy=RESAMPLING_RATIO)  # SMOTE
        else:
            oversampler = ADASYN(sampling_strategy=RESAMPLING_RATIO)  # ADASYN


        X_resampled, y_resampled = oversampler.fit_resample(X, y)

        # Create a DataFrame from the resampled data
        df_all = pd.DataFrame(data=X_resampled, columns=X.columns)
        df_all[classCol] = y_resampled
        return df_all


# Define a function to split a floating-point value and return both parts
def split_float(value):
    try:
        float_value = float(value)
        before_dot = int(float_value)
        after_dot = float_value - before_dot
        return before_dot, after_dot
    except ValueError:
        return None, None

#             My Methods Library
def loggingpreparations(logname='IDSLog.log'):
    #warnings.filterwarnings("ignore")
    logging.basicConfig(filename=logname,
                    filemode=IDSLOG_FILE_MODE,
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S',
                    level=logging.DEBUG)

    logging.info("Running IDS Engine - Classifier Model")
    print("Running IDS Engine - Classifier Model")


def loggingpreparations_OnlyForBestModelTestingPhase(logname='IDSLog_ModelTestingPhase.log'):
    #warnings.filterwarnings("ignore")
    logging.basicConfig(filename=logname,
                    filemode='w',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%d-%b-%y %H:%M:%S',
                    level=logging.DEBUG)

    logging.info("Running IDS Engine - Model Testing Phase - Best DeepNN Classifier Model")
    print("Running IDS Engine - Model Testing Phase - Best DeepNN Classifier Model")

#Recursive To Vigesemal
def tovigisemal(decimalstring):
    dec = int(decimalstring)
    x = (dec % 20)
    digits = "0123456789ABCDEFGHIJ"
    rest = dec / 20
    if (rest == 0):
        return digits[x]
    return tovigisemal(rest) + digits[x]

def sendJobFinishAlertMail(sFrom="ta0002@gmail.com",sTo="ta0002@gmail.com",sSubject="Job Finished",sMessage="Your job is finished!"):
    # Email configuration
    sender_email = sFrom
    receiver_email = sTo
    password = "ywdorcecnqwqmsiz"

    # Create a MIMEText object
    message = MIMEText(sMessage)

    # Set the email subject
    message["Subject"] = sSubject

    # Connect to the SMTP server
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, password)

        # Send the email
        server.sendmail(sender_email, receiver_email, message.as_string())

        # Close the server connection
        server.quit()
        print("Email sent successfully!")

    except Exception as e:
        print(f"Email could not be sent. Error: {str(e)}")

    return

def create_folder_if_not_exists(folder_path):
  # Check if the folder exists
  if not os.path.exists(folder_path):
      # Create the folder if it doesn't exist
      os.makedirs(folder_path)
      print(f"Folder '{folder_path}' created.")
      print('*******************************************************')
      print('')
      print(f"Folder '{folder_path}' created.")
      print('')
  else:
      print(f"Folder '{folder_path}' already exists.")
      print('**************')
      print('')
      print(f"Folder '{folder_path}' already exists.")

  print('*******************************************************')


In [ ]:
sMsg = 'This ML (CIC_IoT2023, K_NO_OF_FEATURES_TO_SELECT = '+str(K_NO_OF_FEATURES_TO_SELECT)+' ,Re-Sampling(Upsampling-SMOTE-Yeo-Johnson-transformation, RATIOs 0.25), also test Unseen 20% data with columns original order'
# Example usage:
print(sMsg)
logging.info('************************************************************')
logging.info('')
logging.info('************************************************************')
logging.info(sMsg)
logging.info('************************************************************')

folder_path = './saved_DNN_models'
create_folder_if_not_exists(folder_path)


This ML (CIC_IoT2023, K_NO_OF_FEATURES_TO_SELECT = 20 ,Re-Sampling(Upsampling-SMOTE-Yeo-Johnson-transformation, RATIOs 0.25), also test Unseen 20% data with columns original order
Folder './saved_DNN_models' created.
*******************************************************

Folder './saved_DNN_models' created.

*******************************************************
